# LangGraph: Zero to Hero Guide

## Building Stateful, Multi-Step AI Agent Workflows

**Objective:** This comprehensive notebook takes you from beginner to advanced LangGraph user. Learn how to build complex, stateful agent workflows with nodes, edges, conditional routing, and human-in-the-loop interactions.

**Target Audience:** Software engineers from complete beginners to experts looking to master LangGraph.

---

## Table of Contents
1. [Introduction & Core Philosophy](#1-introduction--core-philosophy)
2. [Prerequisites & Setup](#2-prerequisites--setup)
3. [Core Concepts: Graphs, Nodes & Edges](#3-core-concepts-graphs-nodes--edges)
4. [Your First Graph](#4-your-first-graph)
5. [State Management](#5-state-management)
6. [Conditional Routing](#6-conditional-routing)
7. [Tool Integration](#7-tool-integration)
8. [Checkpointing & Persistence](#8-checkpointing--persistence)
9. [Human-in-the-Loop](#9-human-in-the-loop)
10. [Subgraphs & Composition](#10-subgraphs--composition)
11. [Multi-Agent Patterns](#11-multi-agent-patterns)
12. [Best Practices & Common Pitfalls](#12-best-practices--common-pitfalls)
13. [Conclusion & Next Steps](#13-conclusion--next-steps)

---

## 1. Introduction & Core Philosophy

### What is LangGraph?

**LangGraph** is a library for building stateful, multi-actor applications with LLMs. It extends LangChain with:

- **Graph-based workflows**: Model complex logic as nodes and edges
- **State management**: Automatic state tracking across nodes
- **Cycles**: Support for loops and iterative refinement
- **Persistence**: Built-in checkpointing for long-running workflows

### Core Philosophy

```
┌─────────────────────────────────────────────────────────────────┐
│                LangGraph Philosophy                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   "Control + Flexibility"                                      │
│                                                                 │
│   • Explicit control flow (unlike autonomous agents)           │
│   • Stateful by design                                         │
│   • Cyclic graphs for iteration                                │
│   • Human-in-the-loop support                                  │
│   • Streaming & persistence built-in                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### When to Use LangGraph?

✅ **Good for:**
- Complex multi-step workflows
- Agents that need explicit control flow
- Human-in-the-loop approval systems
- Long-running tasks with checkpointing
- Multi-agent orchestration

❌ **Consider alternatives when:**
- Simple single-shot tasks (use LangChain)
- Autonomous multi-agent debates (use AutoGen)
- Role-based agent teams (use CrewAI)

### LangGraph vs Other Frameworks

```
┌────────────────────────────────────────────────────────────┐
│              Agent Framework Comparison                    │
├────────────────┬───────────────────────────────────────────┤
│ LangGraph      │ State machines, explicit control flow    │
│ LangChain      │ Single agent, tool calling               │
│ AutoGen        │ Multi-agent conversations                │
│ CrewAI         │ Role-playing agent crews                 │
│ SmolAgents     │ Minimal, code-first agents               │
└────────────────┴───────────────────────────────────────────┘
```

---

## 2. Prerequisites & Setup

### Requirements

- **Python 3.9+**
- **OpenAI API Key**
- **Tavily API Key** (optional, for search examples)

### Installation

```bash
pip install langgraph langchain-openai langchain-community
```

In [ ]:
# Install dependencies (uncomment to run)
# !pip install langgraph langchain-openai langchain-community python-dotenv tavily-python

In [ ]:
import os
import warnings
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Verify API keys
openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

print("🔑 API KEY STATUS")
print("-" * 40)
print(f"OpenAI API Key: {'✅ Found' if openai_key else '❌ Missing'}")
print(f"Tavily API Key: {'✅ Found' if tavily_key else '⚠️ Optional'}")

if not openai_key:
    print("\n❌ Please add OPENAI_API_KEY to your .env file")

---

## 3. Core Concepts: Graphs, Nodes & Edges

LangGraph models workflows as directed graphs:

```
┌─────────────────────────────────────────────────────────────────┐
│                    LangGraph Concepts                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   NODES: Functions that transform state                        │
│   ┌─────────┐    ┌─────────┐    ┌─────────┐                   │
│   │  Node   │───▶│  Node   │───▶│  Node   │                   │
│   │   A     │    │   B     │    │   C     │                   │
│   └─────────┘    └─────────┘    └─────────┘                   │
│                                                                 │
│   EDGES: Connections between nodes                             │
│   - Normal edges: Always follow                                │
│   - Conditional edges: Route based on state                    │
│                                                                 │
│   STATE: Shared data passed between nodes                      │
│   - Defined as TypedDict or Pydantic model                     │
│   - Each node can read and update state                        │
│                                                                 │
│   SPECIAL NODES:                                               │
│   - START: Entry point                                         │
│   - END: Exit point(s)                                         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key Terms

| Term | Description |
|------|-------------|
| **StateGraph** | The graph builder class |
| **Node** | A function that processes state |
| **Edge** | A connection between nodes |
| **Conditional Edge** | Routes based on state values |
| **Checkpoint** | Saved state for resumption |

---

## 4. Your First Graph

Let's build a simple sequential graph.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# Step 1: Define the state schema
class SimpleState(TypedDict):
    """State for our simple graph."""
    messages: list[str]  # List of messages
    counter: int         # A simple counter

print("✅ State schema defined!")

In [ ]:
# Step 2: Define node functions
# Each node receives state and returns updates

def node_a(state: SimpleState) -> dict:
    """First node: Initialize and greet."""
    print("🔵 Node A executing...")
    return {
        "messages": state["messages"] + ["Hello from Node A!"],
        "counter": state["counter"] + 1
    }

def node_b(state: SimpleState) -> dict:
    """Second node: Process and add message."""
    print("🟢 Node B executing...")
    return {
        "messages": state["messages"] + ["Processed by Node B!"],
        "counter": state["counter"] + 1
    }

def node_c(state: SimpleState) -> dict:
    """Third node: Finalize."""
    print("🔴 Node C executing...")
    return {
        "messages": state["messages"] + [f"Completed! Total steps: {state['counter'] + 1}"],
        "counter": state["counter"] + 1
    }

print("✅ Node functions defined!")

In [ ]:
# Step 3: Build the graph

# Create a new graph with our state schema
builder = StateGraph(SimpleState)

# Add nodes
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)
builder.add_node("node_c", node_c)

# Add edges (define the flow)
builder.add_edge(START, "node_a")  # Start -> A
builder.add_edge("node_a", "node_b")  # A -> B
builder.add_edge("node_b", "node_c")  # B -> C
builder.add_edge("node_c", END)  # C -> End

# Compile the graph
simple_graph = builder.compile()

print("✅ Graph compiled!")

In [ ]:
# Step 4: Run the graph

print("\n" + "="*60)
print("🚀 RUNNING GRAPH")
print("="*60 + "\n")

# Initial state
initial_state = {
    "messages": ["Starting workflow..."],
    "counter": 0
}

# Invoke the graph
result = simple_graph.invoke(initial_state)

print("\n" + "="*60)
print("📋 FINAL STATE")
print("="*60)
print(f"Messages: {result['messages']}")
print(f"Counter: {result['counter']}")

### 🎯 Key Takeaways

1. **State** is defined as a TypedDict
2. **Nodes** are functions that transform state
3. **Edges** connect nodes in sequence
4. **START/END** are special nodes
5. **compile()** creates the runnable graph

---

## 5. State Management

LangGraph provides powerful state management with reducers.

In [ ]:
from typing import Annotated
from operator import add

# State with a reducer (accumulates messages)
class AccumulatingState(TypedDict):
    """State that accumulates messages using a reducer."""
    # Annotated with 'add' means new messages are appended, not replaced
    messages: Annotated[list[str], add]
    current_step: str

print("✅ State with reducer defined!")
print("   Using 'add' reducer: messages will accumulate")

In [ ]:
# Nodes for accumulating state

def step_1(state: AccumulatingState) -> dict:
    return {
        "messages": ["Step 1 completed"],
        "current_step": "step_1"
    }

def step_2(state: AccumulatingState) -> dict:
    return {
        "messages": ["Step 2 completed"],
        "current_step": "step_2"
    }

def step_3(state: AccumulatingState) -> dict:
    return {
        "messages": ["Step 3 completed"],
        "current_step": "step_3"
    }

# Build graph
accum_builder = StateGraph(AccumulatingState)
accum_builder.add_node("step_1", step_1)
accum_builder.add_node("step_2", step_2)
accum_builder.add_node("step_3", step_3)
accum_builder.add_edge(START, "step_1")
accum_builder.add_edge("step_1", "step_2")
accum_builder.add_edge("step_2", "step_3")
accum_builder.add_edge("step_3", END)

accum_graph = accum_builder.compile()

print("✅ Accumulating graph compiled!")

In [ ]:
# Run and see accumulation

print("\n" + "="*60)
print("🚀 RUNNING WITH REDUCER")
print("="*60 + "\n")

result = accum_graph.invoke({
    "messages": ["Starting..."],
    "current_step": "init"
})

print("📋 Messages accumulated:")
for i, msg in enumerate(result["messages"]):
    print(f"  {i+1}. {msg}")
print(f"\nFinal step: {result['current_step']}")

---

## 6. Conditional Routing

The real power of LangGraph: dynamic routing based on state.

In [ ]:
# State for conditional routing
class RouterState(TypedDict):
    query: str
    category: str
    response: str

# Classifier node
def classify_query(state: RouterState) -> dict:
    """Classify the query into a category."""
    query = state["query"].lower()
    
    if any(word in query for word in ["weather", "temperature", "rain"]):
        category = "weather"
    elif any(word in query for word in ["calculate", "math", "sum", "multiply"]):
        category = "math"
    else:
        category = "general"
    
    print(f"🏷️ Classified as: {category}")
    return {"category": category}

# Handler nodes
def handle_weather(state: RouterState) -> dict:
    print("🌤️ Weather handler")
    return {"response": f"Weather response for: {state['query']}"}

def handle_math(state: RouterState) -> dict:
    print("🧮 Math handler")
    return {"response": f"Math response for: {state['query']}"}

def handle_general(state: RouterState) -> dict:
    print("💬 General handler")
    return {"response": f"General response for: {state['query']}"}

print("✅ Nodes defined!")

In [ ]:
# Routing function
def route_query(state: RouterState) -> str:
    """Route to the appropriate handler based on category."""
    category = state["category"]
    if category == "weather":
        return "handle_weather"
    elif category == "math":
        return "handle_math"
    else:
        return "handle_general"

# Build conditional graph
router_builder = StateGraph(RouterState)

# Add nodes
router_builder.add_node("classify", classify_query)
router_builder.add_node("handle_weather", handle_weather)
router_builder.add_node("handle_math", handle_math)
router_builder.add_node("handle_general", handle_general)

# Add edges
router_builder.add_edge(START, "classify")

# Add CONDITIONAL edge from classify
router_builder.add_conditional_edges(
    "classify",  # Source node
    route_query,  # Routing function
    {  # Mapping: route_query return -> node name
        "handle_weather": "handle_weather",
        "handle_math": "handle_math",
        "handle_general": "handle_general",
    }
)

# All handlers go to END
router_builder.add_edge("handle_weather", END)
router_builder.add_edge("handle_math", END)
router_builder.add_edge("handle_general", END)

router_graph = router_builder.compile()

print("✅ Conditional router graph compiled!")

In [ ]:
# Test the router with different queries

test_queries = [
    "What's the weather in Paris?",
    "Calculate 15 times 8",
    "Tell me about AI agents"
]

for query in test_queries:
    print("\n" + "="*60)
    print(f"📝 Query: {query}")
    print("="*60)
    
    result = router_graph.invoke({
        "query": query,
        "category": "",
        "response": ""
    })
    
    print(f"📋 Response: {result['response']}")

---

## 7. Tool Integration

LangGraph works seamlessly with LangChain tools.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

# Define tools
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city."""
    weather_data = {
        "paris": "Cloudy, 18°C",
        "london": "Rainy, 12°C",
        "tokyo": "Sunny, 22°C",
    }
    return weather_data.get(city.lower(), f"Unknown weather for {city}")

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except:
        return "Error in calculation"

tools = [get_weather, calculate]

print("✅ Tools defined!")
print(f"   Available: {[t.name for t in tools]}")

In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages

# State for tool-using agent
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize LLM with tools
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Agent node: calls LLM
def agent_node(state: AgentState) -> dict:
    """Call the LLM to decide what to do."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Tool node: executes tools
tool_node = ToolNode(tools)

print("✅ Agent and tool nodes ready!")

In [ ]:
# Router function: continue with tools or end?
def should_continue(state: AgentState) -> str:
    """Decide whether to continue with tools or end."""
    last_message = state["messages"][-1]
    
    # If there are tool calls, route to tools
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    # Otherwise, end
    return "end"

# Build the agent graph
agent_builder = StateGraph(AgentState)

# Add nodes
agent_builder.add_node("agent", agent_node)
agent_builder.add_node("tools", tool_node)

# Add edges
agent_builder.add_edge(START, "agent")
agent_builder.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", "end": END}
)
agent_builder.add_edge("tools", "agent")  # After tools, back to agent

agent_graph = agent_builder.compile()

print("✅ Tool-using agent graph compiled!")

In [ ]:
# Test the agent

print("\n" + "="*60)
print("🚀 TESTING TOOL-USING AGENT")
print("="*60 + "\n")

result = agent_graph.invoke({
    "messages": [HumanMessage(content="What's the weather in Tokyo? Also, calculate 25 * 4")]
})

print("📋 CONVERSATION:")
for msg in result["messages"]:
    if isinstance(msg, HumanMessage):
        print(f"\n👤 Human: {msg.content}")
    elif isinstance(msg, AIMessage):
        if msg.content:
            print(f"\n🤖 AI: {msg.content}")
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"   🔧 Tool calls: {[tc['name'] for tc in msg.tool_calls]}")
    elif isinstance(msg, ToolMessage):
        print(f"   📦 Tool result: {msg.content}")

---

## 8. Checkpointing & Persistence

LangGraph can save state for long-running workflows.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Create a memory checkpointer
checkpointer = MemorySaver()

# Recompile agent with checkpointing
persistent_agent = agent_builder.compile(checkpointer=checkpointer)

print("✅ Agent with checkpointing ready!")
print("   State will be saved between invocations")

In [ ]:
# First conversation turn
config = {"configurable": {"thread_id": "conversation_1"}}

print("\n" + "="*60)
print("💬 TURN 1")
print("="*60 + "\n")

result1 = persistent_agent.invoke(
    {"messages": [HumanMessage(content="What's the weather in Paris?")]},
    config=config
)

# Get the AI response
ai_response = [m for m in result1["messages"] if isinstance(m, AIMessage) and m.content]
if ai_response:
    print(f"🤖 AI: {ai_response[-1].content}")

In [ ]:
# Second conversation turn - uses same thread_id
print("\n" + "="*60)
print("💬 TURN 2 (continues conversation)")
print("="*60 + "\n")

result2 = persistent_agent.invoke(
    {"messages": [HumanMessage(content="How about London?")]},
    config=config  # Same thread_id
)

ai_response = [m for m in result2["messages"] if isinstance(m, AIMessage) and m.content]
if ai_response:
    print(f"🤖 AI: {ai_response[-1].content}")

print(f"\n📊 Total messages in thread: {len(result2['messages'])}")

---

## 9. Human-in-the-Loop

LangGraph supports interrupting for human approval.

In [ ]:
# State for approval workflow
class ApprovalState(TypedDict):
    task: str
    plan: str
    approved: bool
    result: str

def create_plan(state: ApprovalState) -> dict:
    """Create a plan for the task."""
    print("📝 Creating plan...")
    plan = f"Plan for '{state['task']}': Step 1, Step 2, Step 3"
    return {"plan": plan}

def execute_plan(state: ApprovalState) -> dict:
    """Execute the approved plan."""
    print("⚡ Executing plan...")
    return {"result": f"Successfully executed: {state['plan']}"}

def check_approval(state: ApprovalState) -> str:
    """Check if plan is approved."""
    if state.get("approved"):
        return "execute"
    return "wait_for_approval"

print("✅ Approval workflow nodes defined!")

In [ ]:
# Build approval workflow
approval_builder = StateGraph(ApprovalState)

approval_builder.add_node("create_plan", create_plan)
approval_builder.add_node("execute", execute_plan)

approval_builder.add_edge(START, "create_plan")
approval_builder.add_conditional_edges(
    "create_plan",
    check_approval,
    {
        "execute": "execute",
        "wait_for_approval": END  # Stops here for approval
    }
)
approval_builder.add_edge("execute", END)

# Compile with checkpointer for state persistence
approval_checkpointer = MemorySaver()
approval_graph = approval_builder.compile(checkpointer=approval_checkpointer)

print("✅ Approval workflow compiled!")

In [ ]:
# Step 1: Start workflow (will stop at approval)
approval_config = {"configurable": {"thread_id": "approval_1"}}

print("\n" + "="*60)
print("🚀 STEP 1: CREATE PLAN")
print("="*60 + "\n")

result = approval_graph.invoke(
    {"task": "Deploy new feature", "approved": False, "plan": "", "result": ""},
    config=approval_config
)

print(f"📋 Plan created: {result['plan']}")
print("\n⏸️ Workflow paused - waiting for human approval")

In [ ]:
# Step 2: Resume with approval
print("\n" + "="*60)
print("✅ STEP 2: APPROVE AND CONTINUE")
print("="*60 + "\n")

# Update state with approval
approved_state = {**result, "approved": True}

# Create new graph execution with approval
final_result = approval_graph.invoke(
    approved_state,
    config={"configurable": {"thread_id": "approval_2"}}
)

print(f"📋 Result: {final_result['result']}")

---

## 10. Subgraphs & Composition

Build complex workflows by composing smaller graphs.

In [ ]:
# Define subgraph states
class ResearchState(TypedDict):
    topic: str
    findings: str

class WritingState(TypedDict):
    content: str
    draft: str

# Research subgraph
def research_node(state: ResearchState) -> dict:
    print(f"🔍 Researching: {state['topic']}")
    return {"findings": f"Research findings on {state['topic']}"}

research_builder = StateGraph(ResearchState)
research_builder.add_node("research", research_node)
research_builder.add_edge(START, "research")
research_builder.add_edge("research", END)
research_subgraph = research_builder.compile()

# Writing subgraph
def writing_node(state: WritingState) -> dict:
    print(f"✍️ Writing about: {state['content']}")
    return {"draft": f"Draft based on {state['content']}"}

writing_builder = StateGraph(WritingState)
writing_builder.add_node("write", writing_node)
writing_builder.add_edge(START, "write")
writing_builder.add_edge("write", END)
writing_subgraph = writing_builder.compile()

print("✅ Subgraphs defined!")

In [ ]:
# Main graph that uses subgraphs
class MainState(TypedDict):
    topic: str
    findings: str
    draft: str

def do_research(state: MainState) -> dict:
    result = research_subgraph.invoke({"topic": state["topic"], "findings": ""})
    return {"findings": result["findings"]}

def do_writing(state: MainState) -> dict:
    result = writing_subgraph.invoke({"content": state["findings"], "draft": ""})
    return {"draft": result["draft"]}

main_builder = StateGraph(MainState)
main_builder.add_node("research", do_research)
main_builder.add_node("write", do_writing)
main_builder.add_edge(START, "research")
main_builder.add_edge("research", "write")
main_builder.add_edge("write", END)

main_graph = main_builder.compile()

print("✅ Main graph with subgraphs compiled!")

In [ ]:
# Run composed graph
print("\n" + "="*60)
print("🚀 RUNNING COMPOSED WORKFLOW")
print("="*60 + "\n")

result = main_graph.invoke({
    "topic": "AI Agents",
    "findings": "",
    "draft": ""
})

print(f"\n📋 Final Output:")
print(f"  Findings: {result['findings']}")
print(f"  Draft: {result['draft']}")

---

## 11. Multi-Agent Patterns

LangGraph supports various multi-agent coordination patterns.

In [ ]:
# State for multi-agent system
class MultiAgentState(TypedDict):
    task: str
    researcher_output: str
    analyst_output: str
    writer_output: str
    final_report: str

# Agent nodes
def researcher_agent(state: MultiAgentState) -> dict:
    print("🔬 Researcher: Gathering information...")
    return {"researcher_output": f"Research data on {state['task']}"}

def analyst_agent(state: MultiAgentState) -> dict:
    print("📊 Analyst: Analyzing data...")
    return {"analyst_output": f"Analysis of {state['researcher_output']}"}

def writer_agent(state: MultiAgentState) -> dict:
    print("✍️ Writer: Creating report...")
    return {"writer_output": f"Report based on {state['analyst_output']}"}

def supervisor(state: MultiAgentState) -> dict:
    print("👔 Supervisor: Reviewing and finalizing...")
    return {"final_report": f"Final: {state['writer_output']}"}

print("✅ Multi-agent nodes defined!")

In [ ]:
# Build multi-agent graph (pipeline pattern)
multi_builder = StateGraph(MultiAgentState)

multi_builder.add_node("researcher", researcher_agent)
multi_builder.add_node("analyst", analyst_agent)
multi_builder.add_node("writer", writer_agent)
multi_builder.add_node("supervisor", supervisor)

# Pipeline: researcher -> analyst -> writer -> supervisor
multi_builder.add_edge(START, "researcher")
multi_builder.add_edge("researcher", "analyst")
multi_builder.add_edge("analyst", "writer")
multi_builder.add_edge("writer", "supervisor")
multi_builder.add_edge("supervisor", END)

multi_agent_graph = multi_builder.compile()

print("✅ Multi-agent pipeline compiled!")

In [ ]:
# Run multi-agent workflow
print("\n" + "="*60)
print("🚀 RUNNING MULTI-AGENT PIPELINE")
print("="*60 + "\n")

result = multi_agent_graph.invoke({
    "task": "AI Agent Frameworks Comparison",
    "researcher_output": "",
    "analyst_output": "",
    "writer_output": "",
    "final_report": ""
})

print(f"\n📋 FINAL REPORT:")
print(result["final_report"])

---

## 12. Best Practices & Common Pitfalls

### ✅ Best Practices

1. **Define clear state schemas** - Use TypedDict with proper types
2. **Use reducers for lists** - Annotated with add for accumulation
3. **Keep nodes focused** - Single responsibility principle
4. **Add checkpointing** - For long-running workflows
5. **Test subgraphs independently** - Before composing

### ❌ Common Pitfalls

1. **Forgetting END edges** - Graph won't terminate
2. **Missing conditional routes** - Causes runtime errors
3. **State mutation** - Always return new state, don't mutate
4. **Complex routing logic** - Keep routing functions simple
5. **No thread_id for persistence** - Required for checkpointing

In [ ]:
# Production-ready graph template

from typing import TypedDict, Annotated, Literal
from operator import add
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

def create_production_graph():
    """Template for a production-ready LangGraph workflow."""
    
    # 1. Define state with proper types and reducers
    class ProductionState(TypedDict):
        messages: Annotated[list[str], add]
        status: str
        error: str | None
    
    # 2. Define nodes with error handling
    def process_node(state: ProductionState) -> dict:
        try:
            # Processing logic
            return {
                "messages": ["Processed successfully"],
                "status": "completed"
            }
        except Exception as e:
            return {
                "messages": [f"Error: {str(e)}"],
                "status": "error",
                "error": str(e)
            }
    
    # 3. Build graph
    builder = StateGraph(ProductionState)
    builder.add_node("process", process_node)
    builder.add_edge(START, "process")
    builder.add_edge("process", END)
    
    # 4. Add checkpointing
    checkpointer = MemorySaver()
    
    return builder.compile(checkpointer=checkpointer)

print("✅ Production graph template created!")

---

## 13. Conclusion & Next Steps

### What You've Learned

| Topic | Key Takeaway |
|-------|-------------|
| StateGraph | Build workflows as graphs |
| Nodes & Edges | Define processing and flow |
| Conditional Routing | Dynamic path selection |
| Checkpointing | Persist state for resumption |
| Human-in-the-loop | Interrupt for approval |
| Multi-agent | Coordinate multiple agents |

### When to Choose LangGraph

✅ Choose LangGraph when:
- You need explicit control over agent flow
- Workflows require cycles or iteration
- Human approval is required
- Long-running tasks need persistence
- Multi-agent coordination is complex

❌ Consider alternatives when:
- Simple single-shot tasks (LangChain)
- Autonomous agent conversations (AutoGen)
- Role-based agent teams (CrewAI)

### Next Steps

1. **Practice**: Build a workflow for your use case
2. **Explore**: LangGraph Studio for visualization
3. **Deploy**: LangGraph Cloud for production
4. **Compare**: See how other frameworks differ

### Resources

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangGraph GitHub](https://github.com/langchain-ai/langgraph)
- [LangGraph Studio](https://studio.langchain.com/)
- [LangChain Blog](https://blog.langchain.dev/)

---

**Congratulations!** You've completed the LangGraph Zero to Hero guide! 🎉